In [1]:
import pandas as pd
import sys
import gensim, logging
from pymystem3 import Mystem
import wget
import zipfile
from collections import Counter
import pandas as pd

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [3]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/egoluback/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# os.path.getsize('C:\\Users\\vksol\\Downloads\\res2.csv')

In [6]:
train = pd.read_parquet('data_fusion_train.parquet')

In [7]:
train.shape

(26094077, 9)

In [8]:
train.head()

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
0,7,5,09:50,Флуконазол капс 150мг №1 Вертекс,1.0,7,-1,-1,вертекс
1,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,78,None
2,39,4,11:28,"Борщ Станичный с тушенкой, 103 ккал (завод, з/у)",1.0,6,1,-1,None
3,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,71,None
4,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,71,None


In [9]:
udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
model_url = 'http://vectors.nlpl.eu/repository/11/180.zip'
# http://vectors.nlpl.eu/repository/20/212.zip
# model_url = 'http://vectors.nlpl.eu/repository/11/195.zip'

In [11]:
m_ = wget.download(model_url)

KeyboardInterrupt: 

In [10]:
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model_vv = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

2021-03-21 16:17:20,175 : INFO : loading projection weights from <zipfile.ZipExtFile name='model.bin' mode='r' compress_type=deflate>
2021-03-21 16:17:22,721 : INFO : loaded (189193, 300) matrix from <zipfile.ZipExtFile [closed]>


In [11]:
# from simple_elmo import ElmoModel

In [12]:
# # Переписать на безархивную загрузку
# model_url = 'http://vectors.nlpl.eu/repository/11/195.zip'

# model_file = model_url.split('/')[-1]
# with zipfile.ZipFile("Trash/Word2vec/" + model_file, 'r') as archive:
#     model = ElmoModel()
#     model.load("Trash/Word2vec/" +model_file)

In [13]:
train.head()

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
0,7,5,09:50,Флуконазол капс 150мг №1 Вертекс,1.0,7,-1,-1,вертекс
1,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,78,None
2,39,4,11:28,"Борщ Станичный с тушенкой, 103 ккал (завод, з/у)",1.0,6,1,-1,None
3,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,71,None
4,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,71,None


In [14]:
import requests
import re

url = 'https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map'

mapping = {}
r = requests.get(url, stream=True)
for pair in r.text.split('\n'):
    pair = re.sub('\s+', ' ', pair, flags=re.U).split(' ')
    if len(pair) > 1:
        mapping[pair[0]] = pair[1]

print(mapping)
mapping = {'A': 'ADJ', 'ADV': 'ADV', 'ADVPRO': 'ADV', 'ANUM': 'ADJ', 'APRO': 'DET', 'COM': 'ADJ', 'CONJ': 'SCONJ', 'INTJ': 'INTJ', 'NONLEX': 'X', 'NUM': 'NUM', 'PART': 'PART', 'PR': 'ADP', 'S': 'NOUN', 'SPRO': 'PRON', 'UNKN': 'X', 'V': 'VERB'}


def tag_mystem(text='Текст нужно передать функции в виде строки!'):  
    m = Mystem()
    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            if pos in mapping:
                tagged.append(lemma + '_' + mapping[pos]) # здесь мы конвертируем тэги
            else:
                tagged.append(lemma + '_X') # на случай, если попадется тэг, которого нет в маппинге
        except KeyError:
            continue # пропускаем знаки препинания
        except IndexError:
            continue
    return tagged

{'A': 'ADJ', 'ADV': 'ADV', 'ADVPRO': 'ADV', 'ANUM': 'ADJ', 'APRO': 'DET', 'COM': 'ADJ', 'CONJ': 'SCONJ', 'INTJ': 'INTJ', 'NONLEX': 'X', 'NUM': 'NUM', 'PART': 'PART', 'PR': 'ADP', 'S': 'NOUN', 'SPRO': 'PRON', 'UNKN': 'X', 'V': 'VERB'}


In [15]:
# удаляем все, что в скобках, все что 1-2 буквы, все цифры, единицы измерения (например таб)
# подумать над сокращениями
# все что не в модели выкидываем и не думаем (но по хорошему надо бы)
# предлоги частицы и тп, разумеется выкидываем
# если много, то преобразуем в вектор и считаем наиболее релевантное к набору слов ()
# отдельно выделить еду для животных, лекарства, одежда
# В идеале бы почекать все категории

# 500, 6, 2, 505, 656, 700, 755, 1009, 1205
tag_mystem(train.item_name.iloc[1009])

['натуриный_ADJ',
 'пастилка_NOUN',
 'г_NOUN',
 'облепиха_NOUN',
 'с_ADP',
 'витамин_NOUN']

In [16]:
'ланчбаскет_NOUN' in model_vv

False

In [17]:
model_vv.similarity('макароны_NOUN', 'макаронный_ADJ')

0.40270114

In [18]:
train[train.category_id == 79]

2021-03-21 16:17:36,606 : INFO : NumExpr defaulting to 8 threads.


,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
72,131,1,08:21,Спинки варено-копченые из мяса ЦБ в/у охладенные,1.004,12,2,79,None
427,940,1,09:02,ФИЛЕ ЦБ БЕЗ КОЖИ ПОДЛ (ОХЛ),0.758,12,2,79,None
844,1685,4,11:27,Печень говяжья,0.117,15,6,79,None
1040,2056,0,10:25,Котлеты КУРИНЫЕ в панировке ве,0.808,11,2,79,None
1391,2734,4,11:39,ФИЛЕ ЦБ БЕЗ КОЖИ ПОДЛ (ОХЛ),2.999,12,2,79,None
...,...,...,...,...,...,...,...,...,...
45727399,10012217,0,14:33,МЯСО СВИНЫХ ГОЛОВ В ФОРМЕ В/УП ТМ МЯСНОЙ РЯД 1КГ,0.425,14,6,79,None
45727411,10012238,6,09:19,ГОЛЕНЬ ЦЫПЛ-БРОЙ ОХЛ,0.660,11,2,79,None
45728380,10014274,2,19:52,Пакет,2.170,10,2,79,None
45728570,10014684,3,20:38,Набор д/тушения охл подложка Амурский Бройлер,0.928,10,2,79,None


In [19]:
i = 1200
train.iloc[1009:1010]

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
1766,3365,3,16:35,натурино пастилки 36.4г n8 облепиха с витаминами,1.0,8,-1,-1,None


Шоколад с орехами <br />
Изделия макаронные с маслом <br />
С маслом макарошки

In [20]:
# connected_words = ["еда", "овощ", "фрукт", "крупа", "орех", "ягода", "мясо", "рыба", "хлеб", "выпечка"]

In [21]:
model_vv.vector_size

300

In [22]:
train_noreply = train[train.category_id != -1].drop_duplicates('item_name')

In [23]:
from collections import Counter

In [24]:
prepositions = ['в', 'шт', 'кг', 'без', 'до', 'из', 'к', 'на', 'по', 'о', 'от', 'перед', 'при', 'через', 'с', 'у', 'за', 'над', 'об', 'под', 'про', 'для', 'вблизи', 'вглубь', 'вдоль', 'возле', 'около', 'вокруг', 'впереди', 'после', 'посредством', 'путём', 'насчёт', 'поводу', 'ввиду', 'случаю', 'течение', 'благодаря', 'несмотря на', 'спустя']

In [25]:
book = {}

exceptions = {}

for id in train['category_id'].unique():
    df = pd.DataFrame(train_noreply[train_noreply['category_id'] == id]['item_name'])
  
    count = Counter(" ".join(df["item_name"]).lower().split()).most_common(20)
  
    if not count: continue
  
    result = ""
  
    for j in count:
        n = j[0]
        
        n_mystem = tag_mystem(n)
        
        if (not n_mystem): continue
        n_mystem = n_mystem[0]
        
        print("{0} - {1}".format(id, n))
        
        if (len(n) < 3 or n in prepositions or not n.isalpha()): continue
        if (n_mystem not in model_vv):
#             print("{0} - {1}".format(id, n))
            print("word is not defined".format(id, n))
            continue
#             exceptions[id] = n
            
    
        result = n_mystem
    
        break
  
    book[id] = result

frequent = pd.DataFrame.from_dict(book, orient='index')

78 - молоко
71 - с
71 - салат
70 - кофе
84 - с
84 - хлеб
69 - фри
word is not defined
69 - картофель
68 - с
68 - сэндвич
40 - салфетки
203 - упаковка
79 - охл
word is not defined
79 - из
79 - филе
7 - бензин
117 - средство
80 - вес
83 - напиток
53 - брюки
73 - мороженое
81 - чипсы
0 - пиво
12 - масло
85 - чай
77 - соус
145 - корм
2 - сигареты
204 - с
204 - товар
139 - для
139 - зажигалка
38 - таб
74 - колбаса
130 - для
130 - контейнер
75 - ж/б
75 - в
75 - с
75 - ст/б
75 - огурцы
49 - шампунь
45 - ндс)
45 - прием
133 - полотенце
76 - лапша
82 - сельдь
19 - с
19 - открытка
43 - шприц
51 - мыло
61 - футболка
177 - кабель
118 - имп
word is not defined
118 - см
118 - шар
92 - за
92 - содержание
36 - таб
30 - тетрадь
167 - лента
66 - пицца
52 - шапка
107 - по
107 - мм
107 - кисть
37 - детское
3 - стики
word is not defined
3 - табачные
72 - пюре
62 - для
62 - носки
50 - маска
120 - игрушка
42 - для
42 - гель-лак
42 - лак
150 - корм
57 - кроссовки
6 - щетка
140 - семена
101 - батончик
163 - ме

In [26]:
frequent.loc[42]
train[train.category_id == 42]

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
6503,13653,1,11:54,Лак д/ногтей с Кальцием Classics 11мл,1.0,8,1,42,None
10888,23719,5,18:29,Жид-ть д/снятия лака Быстрыйэф-кт30млАсс,2.0,4,1,42,None
23567,51521,4,18:43,"Педикюр ""Пальчики +""",1.0,19,6,42,None
33437,73577,0,21:28,"Маникюр ""Пальчики +""",1.0,18,6,42,None
41018,90418,5,11:11,Лак д/ногтей лечебный 11 мл,1.0,8,6,42,None
...,...,...,...,...,...,...,...,...,...
45711988,9977659,2,16:19,"Зажимы для снятия гель-лака, 5шт ИМП",1.0,8,1,42,None
45723329,10003281,2,11:43,Жид-ть д/снятия лака Быстрыйэф-кт30млАсс,3.0,4,-1,42,None
45723877,10004303,1,20:59,"Маникюр ""Пальчики +""",1.0,18,6,42,None
45727250,10011895,6,19:43,"Маникюр ""Пальчики +""",1.0,18,6,42,None


In [27]:
frequent.loc[118] = tag_mystem("Новогодний")[0]
frequent.loc[79] = tag_mystem("Мясо")[0]
frequent.loc[67] = tag_mystem("Суши")[0]
frequent.loc[13] = tag_mystem("Автомобиль")[0]
frequent.loc[42] = tag_mystem("Косметика")
frequent.loc[9] = tag_mystem("Топливо")[0] # тут, в основном, названия видов топлива, так что word2vec сработает плохо

In [28]:
# for exp_i in exceptions.keys():
#     print("------")
#     print("{0} - {1}".format(exp_i, train_noreply[train_noreply.category_id == exp_i].sample()['item_name']))
#     print("******")
#     print(exceptions[exp_i])

In [29]:
# frequent.loc[69] = tag_mystem("Картофель")[0]
# frequent.loc[79] = tag_mystem("Фарш")[0]
# frequent.loc[118] = tag_mystem("Новогодний")[0]
# frequent.loc[3] = tag_mystem("Табак")[0]
# frequent.loc[67] = tag_mystem("Суши")[0]
# frequent.loc[13] = tag_mystem("Автомобиль")[0]
# frequent.loc[9] = tag_mystem("Топливо")[0] # тут, в основном, названия видов топлива, так что word2vec сработает плохо
# frequent.loc[109] = tag_mystem("Винт")[0]

In [30]:
frequent

,0
78,молоко_NOUN
71,салат_NOUN
70,кофе_NOUN
84,хлеб_NOUN
69,картофель_NOUN
...,...
111,сифон_NOUN
58,платье_NOUN
54,куртка_NOUN
97,ласт_NOUN


In [31]:
from scipy.spatial.distance import cosine

# words = ["еда", "овощ", "фрукт", "крупа", "орех", "ягода", "мясо", "рыба", "хлеб", "выпечка", "макароны", "кофе", "молоко",
#         "напиток", "сыр", "творог", "пакет"
#         ]

# words = frequent[0].values

# conn_w = tag_mystem(" ".join(words))
conn_w = frequent[0].values

def analyze(train, model = model_vv, vec = False):
#     connected_words = tag_mystem(" ".join(words))
    
    connected_words = conn_w
    
    vectors = list(map(model.word_vec, connected_words))
    
    train = delete90(train)
    train = train.apply(lambda x: fin(x, model, vectors, vec))
    return train

def analyze_dict(train, model = model_vv, vec = False):
    connected_words = tag_mystem(" ".join(words))
    
    vectors = list(map(model.word_vec, connected_words))
    
    train = delete90(train)
    train = train.apply(lambda x: fin(x, model, vectors, vec))
    return train


def delete_between_symbols(s1, s2, list_symbol):
    words = (s1, s2)
    if s1 in list_symbol and s2 in list_symbol:
        return list_symbol[:list_symbol.find(words[0]) + len(words[0])-1] + list_symbol[1+list_symbol.rfind(words[1]):]
    else:
        return list_symbol

def delete90(train):
    return train.apply(lambda x: delete_between_symbols('(', ')', x))
                   
def fin(string, model, vectors, vec = False):
    print("------")
    print(string)
    words = tag_mystem(string)
    word = choose_relevant(words, vectors, model)
#     print(word)
    if not vec:
        if word in model:
            return word
        else:
            return 'нет слова'
    else:
        if word in model:
            return model.word_vec(word)
        else:
            return np.array([0] * model.vector_size)
    
    
    
def choose_relevant(words, vectors, model):
    best_word = None
    max_score = -1
    best_pair = None
    ms = 0
#     print(words)
    for i, word in enumerate(words):
#         print("------")
        for v in conn_w:
        #for i, v in enumerate(vectors):
            if word in model:
                #c_d = cosine(model_vv.word_vec(word), v)
                # первые слова получают намного более важное значение, чтобы окончания не отвлекали
                # Тем не менее требуется в любом случае рассматривать все, потому что есть шанс, что сначала идет название
                # Имеет смысл переделать в комбинацию степени и деления. Только степень плохо, ибо появляются проблемы с 1
                # если маленькое сходство, то возможно также стоит проставлять 0, а может мы просто не подумали о таком
                # как вариант, поразбирать автоматически эти кейсы и понять что не так
                c_d = model_vv.similarity(word, v) / (i + 5)
#                 print("{0} with {1}".format(v, c_d * (i + 5)))
                if c_d > max_score:
                    best_word = word
                    max_score = c_d
                    ms = c_d * (i+5)
                    best_pair = v
    print("Best pair is " + str(best_pair) + " with " + str(best_word) + ". Score is " + str(ms))
    
    return best_word

In [32]:
# 10-20 вареный копченый, сработало хорошо

# train_vec = analyze(train.item_name[10:20], vec = True)
train_vec = analyze(train.item_name[:1000], vec = True)
# train_vec = analyze(train.item_name[100:110], vec = True)

train_vec

------
Флуконазол капс 150мг №1 Вертекс
Best pair is None with None. Score is 0
------
Молоко 3,2%,шт
Best pair is молоко_NOUN with молоко_NOUN. Score is 1.0
------
Борщ Станичный с тушенкой, 103 ккал 
Best pair is салат_NOUN with борщ_NOUN. Score is 0.553644597530365
------
Компот из изюма, 114 ккал
Best pair is пюре_NOUN with компот_NOUN. Score is 0.62664794921875
------
Макаронные изделия отварные , 240 ккал
Best pair is товар_NOUN with изделие_NOUN. Score is 0.5481284856796265
------
Филе Горизонт , 218 ккал
Best pair is соус_NOUN with филе_NOUN. Score is 0.6650230884552002
------
3  Бумага Упак 1/Прочи/В асс
Best pair is тетрадь_NOUN with бумага_NOUN. Score is 0.5711789131164551
------
4  Бумага Упак 1/Прочи/В асс
Best pair is тетрадь_NOUN with бумага_NOUN. Score is 0.5711789131164551
------
Брюки спорт мал SAMO M-1016
Best pair is брюки_NOUN with брюки_NOUN. Score is 1.0
------
Бюстгальтер БН137
Best pair is джемпер_NOUN with бюстгальтер_NOUN. Score is 0.6273242235183716
------
Б

Best pair is хлеб_NOUN with сахар_NOUN. Score is 0.535416305065155
------
1/1 Аскорбинка Губернские аптек
Best pair is одеколон_NOUN with аптека_NOUN. Score is 0.3843623399734497
------
Каша овсяная Гудвилл 40г Малина
Best pair is лапша_NOUN with каша_NOUN. Score is 0.5382426977157593
------
Печенье сдобное Удачное вес 
Best pair is вес_NOUN with вес_NOUN. Score is 1.0
------
Рыбное кул.изд. пастер. паста из гиг кал и креветки северной
Best pair is сельдь_NOUN with рыбный_ADJ. Score is 0.5463187098503113
------
*Dinner show 1000*
Best pair is None with None. Score is 0
------
*Домашний с-т с томатами*
Best pair is огурец_NOUN with томат_NOUN. Score is 0.6026803255081177
------
*Драники картоф*
Best pair is блин_NOUN with драник_NOUN. Score is 0.4511116147041321
------
*Матрешка* 250 мл
Best pair is игрушка_NOUN with матрешка_NOUN. Score is 0.4632307291030884
------
*Салат с печенью трески*
Best pair is салат_NOUN with салат_NOUN. Score is 0.9999999403953552
------
*Треска по-польски*
B

Best pair is картофель_NOUN with рис_NOUN. Score is 0.4619128406047821
------
Холодец ХОЛОДУШКА Праздничный 250г
Best pair is лапша_NOUN with холодец_NOUN. Score is 0.5828918814659119
------
Яйцо куриное МЕЖЕНИНОВСКОЕ С1 10шт
Best pair is мясо_NOUN with яйцо_NOUN. Score is 0.47088995575904846
------
.Линейка 15 см Успех, гибкий пластик, ассо
Best pair is набор_NOUN with линейка_NOUN. Score is 0.3649384677410126
------
Гуашь флюорисцентная ЛУЧ 1 баночка 20мл
Best pair is тушь_NOUN with гуашь_NOUN. Score is 0.6729888916015625
------
Карандаши цветные `VK` 18 цв.-Милашки- с заточ.   в карт.короб. с европодвесом
Best pair is тетрадь_NOUN with карандаш_NOUN. Score is 0.5617601871490479
------
Клейкая лента 48*66м. прозрачная
Best pair is лента_NOUN with лента_NOUN. Score is 1.0
------
Краски Гуашь 12 цв. 20ml Hatber-Прикольные коты- в картонной коробке
Best pair is краска_NOUN with краска_NOUN. Score is 1.0
------
Обложка для тетредей ПВХ 120 мкм,проз 213х355
Best pair is папка_NOUN with об

Best pair is соус_NOUN with соус_NOUN. Score is 1.0
------
500Г БЕКОН ВЕНГЕРСКИЙ С/К ДЫМОВ
Best pair is пицца_NOUN with бекон_NOUN. Score is 0.5899410843849182
------
Кукуруза ЛЮТИК отборная ж/б 425мл
Best pair is картофель_NOUN with кукуруза_NOUN. Score is 0.5987171530723572
------
МАСЛО ПОДСОЛНЕЧНОЕ Р/Д ARO 5Л
Best pair is масло_NOUN with масло_NOUN. Score is 1.0
------
Пакет
Best pair is упаковка_NOUN with пакет_NOUN. Score is 0.5296775102615356
------
Соус КУБАНОЧКА по-грузински ст/б 380г
Best pair is соус_NOUN with соус_NOUN. Score is 1.0
------
ФИЛЕ ЦБ БЕЗ КОЖИ ПОДЛ 
Best pair is соус_NOUN with филе_NOUN. Score is 0.6650230884552002
------
Миска 2л син/бостон -103/5
Best pair is лапша_NOUN with миска_NOUN. Score is 0.4603792726993561
------
400Г ФАРШ ДОМАШНИЙ
Best pair is пюре_NOUN with фарш_NOUN. Score is 0.6570708155632019
------
500 ГР., Х."ГЕРКУЛЕС" ЗЕРНОВОЙ НАР
Best pair is пицца_NOUN with геркулес_NOUN. Score is 0.3571096956729889
------
750Г Д/ЧУВС КОШ ИНД ПУРИНА ВАН
Best 

Best pair is молоко_NOUN with вода_NOUN. Score is 0.44815513491630554
------
ЛАЗАНЬЯ ЧЕТЫРЕ СЫРА
Best pair is колбаса_NOUN with сыр_NOUN. Score is 0.6452343463897705
------
Лимон Вес..
Best pair is вес_NOUN with вес_NOUN. Score is 1.0
------
Огурец маленький Приморье
Best pair is огурец_NOUN with огурец_NOUN. Score is 0.9999999403953552
------
ПЕРЧАТКИ ХОЗ-ЫЕ/КД,S
Best pair is куртка_NOUN with перчатка_NOUN. Score is 0.5339561700820923
------
ПРОКЛ.КРИТ.BELLA.КО
Best pair is суша_NOUN with крит_NOUN. Score is 0.24406355619430542
------
ПЯТН АКТ КИСЛ CLEAN
Best pair is содержание_NOUN with акт_NOUN. Score is 0.38109952211380005
------
РАСЧ Д/УКЛ 2 СТОР С
Best pair is лампа_NOUN with стор_NOUN. Score is 0.24559472501277924
------
ТАМП.ГИГ.SUPER+ 8ШТ.
Best pair is None with None. Score is 0
------
Товар   60руб
Best pair is товар_NOUN with товар_NOUN. Score is 1.0
------
- лаваш сырный 2 порц.
Best pair is пицца_NOUN with лаваш_NOUN. Score is 0.5323624014854431
------
Пакет
Best pair is у

...

Best pair is салат_NOUN with салат_NOUN. Score is 0.9999999403953552
------
Суп перловый с грибами 250/7,5
Best pair is соус_NOUN with суп_NOUN. Score is 0.6049665808677673
------
Шея индейки 
Best pair is мясо_NOUN with индейка_NOUN. Score is 0.5633144378662109
------
Контейнер д/анализов, 100 мл ЕЛМД
Best pair is контейнер_NOUN with контейнер_NOUN. Score is 1.0
------
Пакет
Best pair is упаковка_NOUN with пакет_NOUN. Score is 0.5296775102615356
------
Т/мыло Заводъ Братьевъ Крестниковыхъ190г
Best pair is мыло_NOUN with мыло_NOUN. Score is 1.0
------
Набор  тряпок марлев.с рис.0110-3 Н4
Best pair is набор_NOUN with набор_NOUN. Score is 1.0
------
Средство против ржавчины АИСТ Санокс 1100 мл
Best pair is средство_NOUN with средство_NOUN. Score is 1.0
------
Печенье песочно-выемное Орешки вес ООО Золотой колос
Best pair is вес_NOUN with вес_NOUN. Score is 1.0
------
Слойка с ветчиной и сыром 75 гр Прод Лайн
Best pair is колбаса_NOUN with ветчина_NOUN. Score is 0.6645936965942383
------


Best pair is товар_NOUN with продукт_NOUN. Score is 0.6051533222198486
------
Плавленый продукт "ДАЧНЫЙ СЕЗОН" сыросодержащий 70 гр. шт
Best pair is колбаса_NOUN with плавленый_ADJ. Score is 0.5745594501495361
------
Плавленый продукт "ПОПУЛЯРНАЯ ОРБИТА" сыросодержащий 70 гр. шт
Best pair is колбаса_NOUN with плавленый_ADJ. Score is 0.5745594501495361
------
Плавленый продукт "ФИРМЕННАЯ НАДЕЖДА" сыросодержащий 70 гр. шт
Best pair is колбаса_NOUN with плавленый_ADJ. Score is 0.5745594501495361
------
Сыворотка молочная ТМ "Чабан" пастерилизованная" 0%  900 г ПЭТ  TC № RU Д-RU.РА01.В.  Выдан  некомерческая организация "СОЭ
Best pair is молоко_NOUN with сыворотка_NOUN. Score is 0.363282173871994
------
5) Рис отварной 150 гр.
Best pair is соус_NOUN with отварной_ADJ. Score is 0.571746289730072
------
Булочка 8 злаков
Best pair is мороженое_NOUN with булочка_NOUN. Score is 0.5886805653572083
------
Обед Традиционный 1 порц.
Best pair is чай_NOUN with обед_NOUN. Score is 0.5534366369247437


Best pair is салат_NOUN with салат_NOUN. Score is 0.9999999403953552
------
Сметана 10г, 16 ккал
Best pair is соус_NOUN with сметана_NOUN. Score is 0.5661727786064148
------
Суп картофельный с грибами свежими 250г, 100 ккал
Best pair is соус_NOUN with суп_NOUN. Score is 0.6049665808677673
------
гороховое пюре 150г 1 порц.
Best pair is пюре_NOUN with пюре_NOUN. Score is 1.0
------
Бусы на ёлку, 4 м ИМП
Best pair is платье_NOUN with бусы_NOUN. Score is 0.4427119195461273
------
Ветка новогодняя декоративная, 30 см ИМП
Best pair is новогодний_ADJ with новогодний_ADJ. Score is 1.0
------
Пакет
Best pair is упаковка_NOUN with пакет_NOUN. Score is 0.5296775102615356
------
Пакет
Best pair is упаковка_NOUN with пакет_NOUN. Score is 0.5296775102615356
------
Перец Черный горошком 10г Приправыч
Best pair is соус_NOUN with перец_NOUN. Score is 0.5661748051643372
------
Хлеб Семена -злаки 300г
Best pair is хлеб_NOUN with хлеб_NOUN. Score is 0.9999999403953552
------
100Г ГОРЧИЦА ЯДРЕН ТУБА ГЛАВП

Best pair is упаковка_NOUN with пакет_NOUN. Score is 0.5296775102615356
------
Прокладки Bella Perfecta ultra Rose deo frech супертонкие 10 шт
Best pair is кабель_NOUN with прокладка_NOUN. Score is 0.5816521048545837
------
САХАР 900 Г ПЕСОК
Best pair is хлеб_NOUN with сахар_NOUN. Score is 0.535416305065155
------
СОСИСКИ КЛИНСК.460ГР
Best pair is колбаса_NOUN with сосиска_NOUN. Score is 0.7103700041770935
------
ТОМАТЫ КРЫМСКИЕ
Best pair is огурец_NOUN with томат_NOUN. Score is 0.6026803255081177
------
ХЛЕБ РЖЕВ НАР 700ГР
Best pair is хлеб_NOUN with хлеб_NOUN. Score is 0.9999999403953552
------
ЯЙЦО КУРИНОЕ С1 ВАРА
Best pair is мясо_NOUN with яйцо_NOUN. Score is 0.47088995575904846
------
Йогурт фруктовый "Вишня" 2,5%,шт
Best pair is чипсы_NOUN with йогурт_NOUN. Score is 0.6639651656150818
------
Хлеб пшеничный с сыром  чесноком и полентой 500г
Best pair is хлеб_NOUN with хлеб_NOUN. Score is 0.9999999403953552
------
Яблоко Старкинг ЮАР
Best pair is огурец_NOUN with яблоко_NOUN. Scor

0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0.34962243, -0.58396477, 0.059289217, -2.1698...
2       [0.44630435, 2.7040715, -0.5584133, -1.8064618...
3       [-0.57388186, 1.9312656, -0.37914905, -1.16083...
4       [3.7621074, 0.9524328, 0.39624336, 0.68555, -2...
                              ...                        
1740    [1.1112174, -0.08971992, -0.21933037, 0.985152...
1746    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1749    [2.48747, 2.0356464, -1.8317717, -0.8523842, 0...
1750    [-0.039636202, 0.22813235, -0.6793107, -0.0394...
1751    [2.3238804, -0.41672346, 1.2860456, -0.4419828...
Name: item_name, Length: 1000, dtype: object

In [33]:
arr = []
for l in train_vec:
    arr.append(l)
    
np.array(arr).shape

(1000, 300)

In [34]:
train.item_name.loc[1]

'Молоко 3,2%,шт'

In [35]:
d = {}

for i in train_vec.index:
    d[train.item_name.loc[i]] = train_vec.loc[i]

In [36]:
train_vec.index

Int64Index([   0,    1,    2,    3,    4,    6,   10,   11,   12,   13,
            ...
            1726, 1727, 1730, 1737, 1738, 1740, 1746, 1749, 1750, 1751],
           dtype='int64', length=1000)

In [37]:
d[train.item_name.loc[1]]
array = []
for i in train[:1000].item_name.apply(lambda x: d[x]):
    array.append(i)
    
np.array(array).shape

(1000, 300)

In [38]:
pd.DataFrame(array, index = train_vec.index)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.349622,-0.583965,0.059289,-2.169856,-1.829795,-1.010925,-0.983148,1.139235,-0.284853,0.690342,...,2.438885,-2.396471,-1.819215,3.165341,-1.032172,-2.887496,-0.007127,2.266550,-0.859253,1.535934
2,0.446304,2.704072,-0.558413,-1.806462,0.185477,0.639194,0.690984,1.166339,3.735294,-1.817769,...,2.688474,-1.331530,-1.498764,0.738438,-0.627024,-1.051969,-0.088295,-0.278683,-0.048941,-0.486248
3,-0.573882,1.931266,-0.379149,-1.160832,-0.364608,1.320795,-0.336228,2.066413,2.849783,-2.018599,...,1.426697,-2.211262,-1.365813,0.547945,-0.559969,-0.846096,-0.519750,1.274141,0.360176,-0.252460
4,3.762107,0.952433,0.396243,0.685550,-2.771541,1.818107,-0.601072,1.552661,-0.776637,-0.116520,...,-0.692188,-1.667225,-0.071512,-1.061269,1.414781,-2.023448,0.357799,-0.392807,3.549438,-1.990195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1740,1.111217,-0.089720,-0.219330,0.985152,1.320859,2.129070,1.866998,-0.378896,1.024287,-1.791712,...,2.812717,0.778682,-2.466591,1.577921,-2.979939,-1.278097,0.425532,0.642222,1.048818,-1.542141
1746,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1749,2.487470,2.035646,-1.831772,-0.852384,0.995518,-0.504879,-0.998994,-1.623970,1.486557,-0.581432,...,-0.067250,2.169932,0.061856,-0.534442,-1.375227,0.885430,0.762671,1.866933,0.743155,-1.109657
1750,-0.039636,0.228132,-0.679311,-0.039413,0.498102,1.003443,0.025018,-0.634056,0.459712,-1.024626,...,0.710723,-1.260043,-1.445612,0.547612,-0.399701,-0.557464,0.265277,0.657229,0.440621,0.198176


In [39]:
pd.DataFrame(array, index = train_vec.index).to_csv('train_1000.csv')

In [40]:
X = pd.DataFrame(arr, index = train_vec.index)
y = train.category_id[:1000]

In [41]:
from sklearn.model_selection import train_test_split

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [43]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score

In [44]:
clf = CatBoostClassifier(n_estimators=200, depth=3)

clf.fit(X, y)

Learning rate set to 0.308285
0:	learn: 1.4892582	total: 217ms	remaining: 43.2s
1:	learn: 1.3358947	total: 330ms	remaining: 32.7s
2:	learn: 1.2275364	total: 458ms	remaining: 30.1s
3:	learn: 1.1680040	total: 571ms	remaining: 28s
4:	learn: 1.1144040	total: 706ms	remaining: 27.5s
5:	learn: 1.0734826	total: 835ms	remaining: 27s
6:	learn: 1.0436863	total: 956ms	remaining: 26.4s
7:	learn: 1.0218834	total: 1.09s	remaining: 26.2s
8:	learn: 0.9966969	total: 1.31s	remaining: 27.7s
9:	learn: 0.9796962	total: 1.5s	remaining: 28.6s
10:	learn: 0.9659318	total: 1.67s	remaining: 28.7s
11:	learn: 0.9501237	total: 1.83s	remaining: 28.7s
12:	learn: 0.9338232	total: 2.04s	remaining: 29.3s
13:	learn: 0.9256726	total: 2.2s	remaining: 29.2s
14:	learn: 0.9144316	total: 2.36s	remaining: 29.1s
15:	learn: 0.9048546	total: 2.54s	remaining: 29.2s
16:	learn: 0.8981544	total: 2.67s	remaining: 28.7s
17:	learn: 0.8797085	total: 2.77s	remaining: 28s
18:	learn: 0.8582086	total: 2.89s	remaining: 27.5s
19:	learn: 0.847278

162:	learn: 0.4843698	total: 19.1s	remaining: 4.34s
163:	learn: 0.4832896	total: 19.2s	remaining: 4.22s
164:	learn: 0.4811074	total: 19.3s	remaining: 4.1s
165:	learn: 0.4796149	total: 19.4s	remaining: 3.98s
166:	learn: 0.4782204	total: 19.5s	remaining: 3.86s
167:	learn: 0.4771813	total: 19.7s	remaining: 3.75s
168:	learn: 0.4760534	total: 19.9s	remaining: 3.65s
169:	learn: 0.4749656	total: 20s	remaining: 3.53s
170:	learn: 0.4747544	total: 20.1s	remaining: 3.41s
171:	learn: 0.4736036	total: 20.2s	remaining: 3.29s
172:	learn: 0.4734603	total: 20.3s	remaining: 3.17s
173:	learn: 0.4721503	total: 20.4s	remaining: 3.05s
174:	learn: 0.4716423	total: 20.5s	remaining: 2.93s
175:	learn: 0.4707047	total: 20.6s	remaining: 2.81s
176:	learn: 0.4691942	total: 20.8s	remaining: 2.7s
177:	learn: 0.4685427	total: 21s	remaining: 2.59s
178:	learn: 0.4675795	total: 21.2s	remaining: 2.48s
179:	learn: 0.4670623	total: 21.3s	remaining: 2.36s
180:	learn: 0.4661554	total: 21.4s	remaining: 2.24s
181:	learn: 0.4658

In [45]:
import pickle
pickle.dump(model_vv, open('model_vv', 'wb'))

In [46]:
model = pickle.load(open('model_vv', 'rb'))

In [49]:
X_train.to_csv('xtrain.csv')
y_train.to_csv('ytrain.csv')

X_test.to_csv('xtest.csv')
y_test.to_csv('ytest.csv')

In [50]:
from sklearn.metrics import f1_score, accuracy_score

In [51]:
X_train = pd.read_csv("xtrain.csv")

In [54]:
f1_score(y_train, clf.predict(X_train), average = 'weighted')

0.7434759769511502

In [55]:
f1_score(y_test, clf.predict(X_test), average = 'weighted')

0.7873063895304079

In [57]:
accuracy_score(y_train, clf.predict(X_train))

0.8053333333333333

In [58]:
accuracy_score(y_test, clf.predict(X_test))

0.844

In [154]:
model_vv.similarity("суп_NOUN", "компот_NOUN")

0.61981076

In [155]:
model_vv.most_similar("компот_NOUN")

2021-03-21 15:40:48,593 : INFO : precomputing L2-norms of word weight vectors


[('супчик_NOUN', 0.673333466053009),
 ('варенье_PROPN', 0.6702350378036499),
 ('джем_NOUN', 0.647370457649231),
 ('варенье_NOUN', 0.6456072330474854),
 ('морс_NOUN', 0.6420105695724487),
 ('запеканка_NOUN', 0.630162239074707),
 ('пюре_NOUN', 0.6266480684280396),
 ('какао_PROPN', 0.6259390115737915),
 ('омлет_NOUN', 0.6223021745681763),
 ('суп_NOUN', 0.6198108196258545)]

In [156]:
train.item_name.iloc[:10]

0                      Флуконазол капс 150мг №1 Вертекс
1                                        Молоко 3,2%,шт
2      Борщ Станичный с тушенкой, 103 ккал (завод, з/у)
3                             Компот из изюма, 114 ккал
4     Макаронные изделия отварные (масло сливочное),...
6                  Филе Горизонт (филе птицы), 218 ккал
10                         3  Бумага Упак 1/Прочи/В асс
11                         4  Бумага Упак 1/Прочи/В асс
12                          Брюки спорт мал SAMO M-1016
13                                    Бюстгальтер БН137
Name: item_name, dtype: object

In [157]:
a

NameError: name 'a' is not defined

In [ ]:
delete90(train.item_name)

In [16]:
train.head()

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
1,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,78,None
3,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,71,None
4,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,71,None
17,56,5,11:42,Кофе Капучино Большой Эден 18,1.0,12,1,70,None
40,105,3,01:53,Хлеб на СЫВОРОТКЕ 350г,1.0,7,-1,84,None


In [29]:
delete_between_symbols('(', ')', train.item_name.iloc[2])

'Макаронные изделия отварные , 240 ккал'

In [7]:
train['category_id'].value_counts()

84     7094
71     4771
78     2869
83     2865
0      2356
       ... 
102      19
101      17
46       15
100      14
97       13
Name: category_id, Length: 96, dtype: int64

In [20]:
train[train['category_id'] == 84]['item_nds_rate'].value_counts()

 6    2468
 1    2309
 2    1229
-1     838
 5     167
 3      59
 4      24
Name: item_nds_rate, dtype: int64

In [27]:
train['item_nds_rate'].value_counts()

1           2
3           1
4           1
17          1
40         -1
           ..
45669181    6
45681543    6
45690702    6
45692298    6
45700308    6
Name: item_nds_rate, Length: 48225, dtype: int32

In [28]:
train['brands'].value_counts()

орбит                  200
кока-кола              189
пепси                  118
вискас                  81
лейс                    71
                      ... 
айдиго                   1
donella                  1
маккорн                  1
marko                    1
деревенский самогон      1
Name: brands, Length: 2504, dtype: int64

In [24]:
train.head()

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
1,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,78,None
3,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,71,None
4,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,71,None
17,56,5,11:42,Кофе Капучино Большой Эден 18,1.0,12,1,70,None
40,105,3,01:53,Хлеб на СЫВОРОТКЕ 350г,1.0,7,-1,84,None


In [2]:
train = pd.read_csv('data_fusion_train.csv', index_col= 0)
train = train[train.category_id != -1].drop_duplicates('item_name')

In [4]:
train = train.sample(1000)

In [19]:
(train != train).sum(axis = 0)

receipt_id               0
receipt_dayofweek        0
receipt_time             0
item_name                0
item_quantity            0
item_price               0
item_nds_rate            0
category_id              0
brands               40816
dtype: int64

In [13]:
train.head()

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
1,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,78,None
3,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,71,None
4,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,71,None
17,56,5,11:42,Кофе Капучино Большой Эден 18,1.0,12,1,70,None
40,105,3,01:53,Хлеб на СЫВОРОТКЕ 350г,1.0,7,-1,84,None


In [10]:
stop = stopwords.words('russian')
tfidf = TfidfVectorizer(stop_words=stop, max_features=1000)
X_train = tfidf.fit_transform(train.item_name)

In [11]:
X_train

<48225x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 105161 stored elements in Compressed Sparse Row format>

In [36]:
train..shape

(9, 48225)

In [37]:
X_train.T.shape

(1000, 48225)

In [49]:
c = X_train.toarray()

In [60]:
np.concatenate([train.T, c.T]).shape

(1009, 48225)

In [48]:

df = pd.DataFrame({'index': c.row, 'node2': c.col, 'edge_weight': c.data})

In [56]:
df = pd.DataFrame(np.concatenate([train.T, c.T]))

In [63]:
df = df.T

In [64]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,999,1000,1001,1002,1003,1004,1005,1006,1007,1008
0,11,6,20:34,"Молоко 3,2%,шт",2,8,2,78,None,0,...,0,0,0,0,0,0,0,0,0,0
1,39,4,11:28,"Компот из изюма, 114 ккал",1,4,1,71,None,0,...,0,0,0,0,0,0,0,0,0,0
2,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1,4,1,71,None,0,...,0,0,0,0,0,0,0,0,0,0
3,56,5,11:42,Кофе Капучино Большой Эден 18,1,12,1,70,None,0,...,0,0,0,0,0,0,0,0,0,0
4,105,3,01:53,Хлеб на СЫВОРОТКЕ 350г,1,7,-1,84,None,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
df.to_csv('data_fusion_train_concatAll.csv')

In [67]:
df = df.drop(3, axis = 1)

In [70]:
y = df[7]
df = df.drop([2, 8, 7], axis = 1)

In [6]:
clf = LogisticRegression(max_iter=400)
# cross_val_score(clf, df, y, cv=3, scoring='f1_weighted')

In [71]:
from sklearn.model_selection import train_test_split
X_train1, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=42)

In [7]:
clf.fit(X_train, train.category_id)

LogisticRegression(max_iter=400)

In [32]:
from sklearn.metrics import f1_score

In [9]:
f1_score(train.category_id, clf.predict(X_train), average = 'weighted')

0.46825497858760295

In [10]:
model = CatBoostClassifier(depth = 4, num_trees = 15)

In [8]:
df = pd.read_csv('data_fusion_train_concatAll.csv', index_col = 0)
y = df['7']
df = df.drop(['2', '8', '7', '3'], axis = 1)

X_train1, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=42)

In [ ]:

model.fit(X_train, train.category_id)

In [12]:
f1_score(y_test, model.predict(X_test), average = 'weighted')

0.12790761675391094

In [8]:
import pickle
pickle.dump(tfidf, open('tfidf', 'wb'))
pickle.dump(model, open('clf_task1', 'wb'))